In [2]:
#load packages
import numpy as np
import tensorflow as tf
from tensorflow import keras 
import pandas as pd
import soundfile as sf

In [3]:
#load the saved model 
json_file = open('NN_216-36-5.json', 'r')
loaded_model_json = json_file.read()
loaded_model = keras.models.model_from_json(loaded_model_json)

#load weights
loaded_model.load_weights("NN_216-36-5_weights.h5")

In [4]:
#load test data
path = 'D:/Processed/'#'x:/Steering/Recordings/Processed/'
testX = np.load(path + 'TestX_NN_216-36-5.npy')
testY = np.load(path + 'TestY_NN_216-36-5.npy')
one_hot_test = np.load(path + 'OneHot_testY_216-36-5.npy')

In [5]:
testY

array([2, 2, 5, ..., 4, 3, 5], dtype=int64)

In [6]:
loaded_model.compile(loss='mean_squared_error',
    optimizer='Adam',
           metrics=['categorical_accuracy'])

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [7]:
loaded_model.evaluate(testX,one_hot_test,verbose=0)

[0.04227109306540517, 0.860501656217642]

In [8]:
#process weights
weights_first_layer = loaded_model.layers[0].get_weights()[0]
biases_first_layer = loaded_model.layers[0].get_weights()[1]

weights_second_layer = loaded_model.layers[1].get_weights()[0]
biases_second_layer = loaded_model.layers[1].get_weights()[1]

In [9]:
weights_first_layer = np.vstack([biases_first_layer, weights_first_layer])
weights_second_layer = np.vstack([biases_second_layer, weights_second_layer])

#tmp1 = abs(np.vstack([weights_first_layer, biases_first_layer])).max()
#tmp2 = abs(np.vstack([weights_second_layer, biases_second_layer])).max()

In [10]:
weights_first_layer_norm = weights_first_layer / abs(weights_first_layer).max()
#biases_first_layer_norm = biases_first_layer / abs(biases_first_layer).max()

weights_second_layer_norm = weights_second_layer / abs(weights_second_layer).max()
#biases_second_layer_norm = biases_second_layer / tmp2 #abs(biases_second_layer).max()

In [11]:
abs(weights_first_layer).max(), abs(weights_second_layer).max()

(131.29463, 21.670479)

In [12]:
weights_first_layer.shape, weights_second_layer.shape

((217, 36), (37, 5))

In [13]:
np.savetxt('weights_first_layer_norm_NN_216-36-5.txt',
           weights_first_layer_norm,delimiter= ',')
np.savetxt('weights_second_layer_norm_NN_216-36-5.txt',
           weights_second_layer_norm,delimiter= ',')
